# <div align="center" style="color: #ff5733;">TSA Account Closure Third Notification</div>

In [4]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [5]:
sq = """select Run_Date, count(1)cnt from risk_credit_mis.tsa_account_closure group by 1 order by 1 desc"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
df.head(1)

Job ID a4ecd245-5dfd-42e3-87da-fbb2e4f345df successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,Run_Date,cnt
0,2024-04-23,538727


The date should show yesterday's date

# Read the data from the first sheet for Type1 and Type2 data provided by Marvin

In [6]:
d1 = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\Feb24  & March24 Ongoing Batch\Type 1 & 2 - 2nd Notification20240423.xlsx")
d1.shape

(6795, 2)

# Read the data from the second sheet for Type3 data provided by Marvin

In [7]:
print(f"The unique count of customer_id is:\t{len(d1['Customer_Id'].unique())}")
d1.head()
d2 = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\Feb24  & March24 Ongoing Batch\Type 3 - 2nd Notification20240423.xlsx")
d2.shape

The unique count of customer_id is:	6795


(1305, 2)

In [8]:
print(f"The unique count of customer_id is:\t{len(d2['Customer_Id'].unique())}")
d2.head()

The unique count of customer_id is:	1305


,Customer_Id,ACTION_TYPE
0,2121773,TYPE 3
1,2155674,TYPE 3
2,1879666,TYPE 3
3,2226117,TYPE 3
4,2171014,TYPE 3


In [9]:
result = pd.concat([d1, d2], axis=0, ignore_index=True)
result.shape

(8100, 2)

In [10]:
result.ACTION_TYPE.value_counts()

ACTION_TYPE
TYPE 1    6771
TYPE 3    1305
TYPE 2      24
Name: count, dtype: int64

In [11]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8100 entries, 0 to 8099
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_Id  8100 non-null   int64 
 1   ACTION_TYPE  8100 non-null   object
dtypes: int64(1), object(1)
memory usage: 126.7+ KB


In [12]:
result['Customer_Id'] = result['Customer_Id'].astype(str)
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8100 entries, 0 to 8099
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_Id  8100 non-null   object
 1   ACTION_TYPE  8100 non-null   object
dtypes: object(2)
memory usage: 126.7+ KB


In [13]:
a = tuple(result['Customer_Id'])
a[:5]

('2351977', '2359481', '2284746', '2369297', '2322506')

The date should be D-1? If the date is not D-1 then ask Bala to refresh the table.

Step1: At line 11 change the date to D-1<br><br>
Step2: At line around 36 (and (TSA_Opening_date < date '2023-10-18')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step3: At line around 45 (WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '2023-10-18')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step4: At line 58 (and TSA_Opening_date < date '2023-06-18'  and (Last_TSA_Credit_Date < date '2023-06-18' and Last_TSA_Debit_Date < date '2023-06-18' and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '2023-06-18')  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) <br><br>
Step5: At line 72 (Run_Date = date '2024-01-09'###DATE change here to D-1) Change the date to D-1<br><br>

First Notification we change all the steps <br><br>
Second and Third Notification we only change Step1 and Step5
For Second Notificaiton and Third Notificaiton the customre id should be from the file sent by Marvin. <br><br>

In [14]:
sq = f"""
with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '2024-04-23'  ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id in {a}
),
w_approved
as
(
  select DISTINCT customerId from risk_credit_mis.loan_master_table where loantype = 'BIGLOAN' OR FLAGAPPROVAL = 1 OR FLAGDISBURSEMENT = 1  
  or (stage1Status = 'EXPIRED' AND cddInitiatedDateAndTime IS NULL)
)
select *
from
(
  select distinct
  a.Customer_Id,a.TSA_Opening_date,
  c.OFSTANDARDACCOUNTID as TSA_Account_Number, c.OfblockAmount, 
  a.TSA_Status,a.TSA_Balance,a.Solo_Stash_Balance,a.Group_Stash_Balance,a.Term_Deposit_Balance,a.Active_Term_Deposit_flag,a.Last_TSA_Credit_Date,a.Last_TSA_Debit_Date,
  a.ARI_Flag,a.Active_Loan,a.Total_Loan_Outstanding,  case when b.customerId is not null then 1 else 0 end as Approved_Loans,
  a.Pending_Loan_Applications,a.Hard_Reject_Loans_App,a.Soft_Reject_Loans_App,a.Cancelled_Loans_App, extract(date from a.Last_Login_Date) as Last_Login_Date,
  a.Last_TSA_EOD_Balance_Date,a.Last_Group_Stash_Credit_Date,a.Group_Stash_Balance_Owner,a.Run_Date,a.Batch,a.Processing_Month,a.pending_loans_not_expired_not_exempt,a.expired_loans,
  a.pending_loans_lastDt,a.expired_loans_lastDt,
  CASE 
    WHEN tsa_status = 'Active' and (Last_TSA_Credit_Date = date '1970-01-01' and Last_Group_Stash_Credit_Date is null)
    and (TSA_Opening_date < date '2024-01-25')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)  THEN 'TYPE 1'
    and b.customerId is null  THEN 'TYPE 1'

    WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '2024-01-25')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
    and b.customerId is null  
    and (date_trunc(TSA_Opening_date,day) = date_trunc(Last_Login_Date,day)) THEN 'TYPE 2'

    
    WHEN ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) < 1 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and TSA_Opening_date < date '2023-09-27'  
    and (Last_TSA_Credit_Date < date '2023-09-27' 
    and Last_TSA_Debit_Date < date '2023-09-27'
    and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01')  < date '2023-09-27')  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and tsa_status = 'Active'
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
    and b.customerId is null  
    THEN 'TYPE 3'
    ELSE 'NO ACTION'
    END AS ACTION_TYPE
  from w_tsa A
  left join w_approved b
  on cast(a.Customer_Id as int64) = b.customerId
  left join core_raw.customer_accounts c
  on c.OFCUSTOMERID = a.Customer_Id and c.CRINTERDESC = 'Transactional Savings Account Inv_R' and c.OFISCLOSED = 'N'
  where
  Run_Date = date '2024-04-23'###DATE change here to D-1
)
"""


In [15]:
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 3da177a3-0e8d-4b39-b8c1-38af88ad1b2c successfully executed: |          |
Downloading: 100%|██████████|


In [16]:
pd.set_option('display.max_columns', None)
df.sample(10)

,Customer_Id,TSA_Opening_date,TSA_Account_Number,OfblockAmount,TSA_Status,TSA_Balance,Solo_Stash_Balance,Group_Stash_Balance,Term_Deposit_Balance,Active_Term_Deposit_flag,Last_TSA_Credit_Date,Last_TSA_Debit_Date,ARI_Flag,Active_Loan,Total_Loan_Outstanding,Approved_Loans,Pending_Loan_Applications,Hard_Reject_Loans_App,Soft_Reject_Loans_App,Cancelled_Loans_App,Last_Login_Date,Last_TSA_EOD_Balance_Date,Last_Group_Stash_Credit_Date,Group_Stash_Balance_Owner,Run_Date,Batch,Processing_Month,pending_loans_not_expired_not_exempt,expired_loans,pending_loans_lastDt,expired_loans_lastDt,ACTION_TYPE
6236,2339397,2024-01-16,60823393970001,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,1,0,0,2024-01-16,NaT,NaT,NaN,2024-04-23,Batch_1,April,0,0,NaT,NaT,TYPE 1
5460,2366288,2024-01-14,60823662880001,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,1,0,0,2024-01-14,NaT,NaT,NaN,2024-04-23,Batch_1,April,0,0,NaT,NaT,TYPE 1
1088,2360775,2024-01-06,60823607750006,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-01-06,NaT,NaT,NaN,2024-04-23,Batch_1,April,0,1,NaT,2024-04-06,TYPE 1
3735,2378865,2024-01-24,60823788650004,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-02-17,NaT,NaT,NaN,2024-04-23,Batch_1,April,0,0,NaT,NaT,TYPE 1
8035,2355175,2023-12-31,60823551750003,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-01-22,NaT,NaT,NaN,2024-04-23,Batch_1,April,0,1,NaT,2024-03-31,TYPE 1
2294,2374509,2024-01-20,60823745090001,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-04-20,NaT,NaT,NaN,2024-04-23,Batch_1,April,0,1,NaT,2024-04-20,TYPE 1
1561,2163492,2023-07-30,60821634920008,0.0,Active,0.01,0.0,0.0,0.0,0,2023-08-31,2023-08-02,N,0,0.0,0,0,0,0,0,2023-08-02,2024-04-23,NaT,NaN,2024-04-23,Batch_1,April,0,0,NaT,NaT,TYPE 3
2434,2368887,2024-01-14,60823688870005,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-01-17,NaT,NaT,NaN,2024-04-23,Batch_1,April,0,1,NaT,2024-04-14,TYPE 1
4429,2160486,2023-08-03,60821604860008,0.0,Active,0.02,0.0,0.0,0.0,0,2023-08-31,2023-08-07,N,0,0.0,0,0,1,0,0,2023-08-07,2024-04-23,NaT,NaN,2024-04-23,Batch_1,April,0,0,NaT,NaT,TYPE 3
6862,2364892,2024-01-15,60823648920006,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-01-15,NaT,NaT,NaN,2024-04-23,Batch_1,April,0,1,NaT,2024-04-10,TYPE 1


In [17]:
df.shape

(8097, 32)

In [18]:
df.to_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\Feb24  & March24 Ongoing Batch\20240424TSA_Account_Closure_3rdtNotification.xlsx", sheet_name='TSA_Closure_3rdNotification', index = False)